In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
wallets = pd.read_csv("wallets_cleaned.csv")

In [ ]:
wallets["rank"] = wallets.index + 1

In [ ]:
allocations = [0.22, 0.48, 0.2, 0]
slopes = [0.0125, 0.002, 0.00105, 0]
counts = [96, 852, 4052, np.inf]

In [ ]:
[0, sum(counts[0:1]), sum(counts[0:2]), sum(counts[0:2], len(wallets))],
cum_counts = [0] + [sum(counts[:i]) for i in range(1, len(counts) + 1)]
weights = [(((1 + slopes[i]) ** counts[i]) - 1) / slopes[i] if slopes[i] != 0 else 1 for i in range(len(slopes))]

In [ ]:
wallets["tier"] = pd.cut(wallets["rank"], bins=cum_counts, labels=range(1, len(counts) + 1))

In [ ]:
wallets["allocation"] = wallets.apply(lambda row: (1 + slopes[row["tier"] - 1]) ** (cum_counts[row["tier"]] - row["rank"]) * allocations[row["tier"] - 1] / weights[row["tier"] - 1], axis=1)

In [ ]:
total_bridged = wallets["JOE_BRIDGED"].sum()
wallets["ratio"] = wallets["allocation"] / (wallets["JOE_BRIDGED"] / total_bridged)

In [ ]:
ranks = wallets["rank"]
allocations = np.log(wallets["allocation"])

plt.plot(ranks, allocations)
plt.show()

In [ ]:
ranks = wallets["rank"]
ratios = np.log(wallets["ratio"])

plt.plot(ranks, ratios)

In [ ]:
for tier in range(1, len(counts)):
    print(f"Tier {tier}: ({cum_counts[tier-1]}, {cum_counts[tier] - 1})")
    print(f"Allocation: {wallets['allocation'].loc[cum_counts[tier-1]]}, {wallets['allocation'].loc[cum_counts[tier] - 1]}")